In [1]:
# Импорт всех необходимых модулей для оптимизации
import sys
import os
from pathlib import Path

# Добавляем путь к модулям проекта
project_root = Path.cwd()
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Основные модули оптимизации
from optimization import (
    # Основные классы
    OptimizationExecutor,
    OptimizationReporter,
    
    # Хранилище результатов
    InMemoryResultsStore,
    OptimizationResult,
    ResultsStoreProtocol,
    
    # Пространство параметров
    ParameterDefinition,
    ParameterSpace,
    
    # Стратегии поиска
    SearchStrategy,
    GridSearchStrategy,
    RandomSearchStrategy,
)

# Дополнительные импорты для работы с данными и бэктестингом
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Optional, Mapping, Sequence
from dataclasses import dataclass
from datetime import datetime, UTC

# Импорты для работы с сигналами и индикаторами
from signals.base_signal import BaseSignal
from signals.signal_registry import SignalRegistry
from indicators.base_indicator import BaseIndicator
from indicators.indicator_registry import IndicatorRegistry

# Импорты для бэктестинга
from backtest.engine import BacktestEngine
from backtest.metrics import MetricsCalculator
from backtest.trade import Trade
from backtest.commission import CommissionCalculator
from optimization.executor import SignalConfig

print("✅ Все модули оптимизации успешно импортированы!")
print(f"📁 Рабочая директория: {project_root}")
print(f"📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy")
print(f"📊 Доступные хранилища: InMemoryResultsStore")
print(f"📈 Доступные репортеры: OptimizationReporter")


✅ Все модули оптимизации успешно импортированы!
📁 Рабочая директория: c:\Users\user\Documents\spreader_pro\code\trend_optimization
📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy
📊 Доступные хранилища: InMemoryResultsStore
📈 Доступные репортеры: OptimizationReporter


In [12]:
# Ячейка 1: Импорты и настройка
import sys
sys.path.append('src')
from data import CSVLoader
import pandas as pd
import numpy as np

from lightweight_charts import JupyterChart
# Ячейка 2: Настройка данных
DATA_DIR = r"G:\My Drive\data_fut"
TICKER = "IMOEXF"
loader = CSVLoader(DATA_DIR, use_cache=True)

# Ячейка 3: Загрузка данных
start_date = '2025-01-15'
end_date = '2025-10-15'

df = loader.load(TICKER, timeframe=5, start_date=start_date, end_date=end_date)
print(f"Данные: {len(df)} баров, период: {df.index[0]} - {df.index[-1]}")

# Ячейка 4: Ваш анализ (например, Volatility Median)
# Здесь можете использовать функции из VM_base_optimize.py

Данные: 35569 баров, период: 2025-01-15 10:00:00 - 2025-10-14 23:50:00


In [14]:
# Простая функция вместо класса SignalFactory
def create_signal_config(parameters):
    """Разделяем параметры по назначению"""
    return SignalConfig(
        signal=SignalRegistry.get("SlopeSignal"),
        signal_params={'threshold': 0.0001},  # SlopeSignal: threshold, absolute
        indicator_params={
            'KATR': parameters.get('katr'),
            'PerATR': parameters.get('peratr'),
            'SMA': parameters.get('sma'),
            'MinRA': parameters.get('minra'),
            'FlATR': parameters.get('flatr'),
            'FlHL': parameters.get('flhl')
        },
        backtest_params={
            # entry_price_type/n_contracts можно переопределить здесь при необходимости
            # 'entry_price_type': 'close',
            # 'n_contracts': 1,
        }
    )

# Простая функция вместо класса IndicatorBuilder
def build_indicator(df, params, *, verbose=False):
    """Создаем индикатор с параметрами"""
    vm_ind = IndicatorRegistry.get("VolatilityMedian")

    # Параметры для VolatilityMedian: KATR, PerATR, SMA, MinRA, FlATR, FlHL
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL'
    }

    indicator_params = {}
    for param_key, param_value in (params or {}).items():
        normalized_key = str(param_key).lower()
        mapped_key = param_mapping.get(normalized_key)
        if mapped_key and param_value is not None:
            indicator_params[mapped_key] = param_value

    if verbose:
        print(f"Параметры для индикатора: {indicator_params}")

    return vm_ind.calculate(df, **indicator_params)


In [15]:
# Простой способ - создание из ваших данных
your_params = {
    "KATR": [2, 3, 4, 5, 6, 7, 8],
    "PerATR": [5, 20, 80], 
    "SMA": [1, 2, 3, 10],
    "MinRA": [0],
    "FlATR": [0],
    "FlHL": [0]
}

# Преобразование в ParameterSpace
definitions = []
for name, values in your_params.items():
    definitions.append(ParameterDefinition(
        name=name.lower(),  # приводим к нижнему регистру
        values=values
    ))

parameter_space = ParameterSpace.from_definitions(definitions)

In [16]:
# Создаем оптимизатор
executor = OptimizationExecutor(
    backtest_engine=BacktestEngine(),
    parameter_space=parameter_space,
    strategy=GridSearchStrategy(parameter_space),
    signal_factory=create_signal_config,  # ← Простая функция!
    results_store=InMemoryResultsStore(),
    indicator_builder=build_indicator,    # ← Простая функция!
    # ↓↓↓ новые настройки производительности/стабильности ↓↓↓
    checkpoint_interval=1000,              # было 1 → станет реже и быстрее
    per_candidate_timeout=None,           # убрать пер-кандидатные пулы
    default_max_workers=1,                # дефолт: одиночный режим
    checkpoint_enabled=True               # оставить включённым (но с редким интервалом)
    
)

print("✅ Оптимизатор создан!")

✅ Оптимизатор создан!


In [17]:
# === РАСЧЕТЫ ПЕРЕД ЗАПУСКОМ ===
print("📊 АНАЛИЗ ПАРАМЕТРОВ ОПТИМИЗАЦИИ")
print("=" * 50)

# 1. Размер данных
print(f"📈 Будет протестировано: {len(df):,} свечей")
print(f"📅 Период данных: {df.index[0].strftime('%Y-%m-%d')} - {df.index[-1].strftime('%Y-%m-%d')}")

# 2. Расчет количества комбинаций
total_combinations = 1
param_details = []

for name, values in your_params.items():
    count = len(values)
    total_combinations *= count
    param_details.append(f"  {name}: {count} значений {values}")

print(f"\n🔢 ПАРАМЕТРЫ ОПТИМИЗАЦИИ:")
for detail in param_details:
    print(detail)

print(f"\n🧮 РАСЧЕТ КОМБИНАЦИЙ:")
print(f"  Всего параметров: {len(your_params)}")
print(f"  Комбинаций: {total_combinations:,}")
print(f"  Формула: {' × '.join([str(len(values)) for values in your_params.values()])} = {total_combinations:,}")

# 3. Оценка времени выполнения
estimated_time_per_combination = 0.1  # секунд на комбинацию (примерно)
total_time_seconds = total_combinations * estimated_time_per_combination
total_time_minutes = total_time_seconds / 60
total_time_hours = total_time_minutes / 60

print(f"\n⏱️ ОЦЕНКА ВРЕМЕНИ:")
print(f"  ~{estimated_time_per_combination} сек на комбинацию")
print(f"  Общее время: {total_time_seconds:.1f} сек ({total_time_minutes:.1f} мин)")
if total_time_hours >= 1:
    print(f"  Или: {total_time_hours:.1f} часов")

# 4. Предупреждения
if total_combinations > 10000:
    print(f"\n⚠️ ВНИМАНИЕ: Большое количество комбинаций!")
    print(f"   Рекомендуется уменьшить параметры или использовать RandomSearchStrategy")
    
if total_combinations > 100000:
    print(f"\n🚨 КРИТИЧНО: Очень много комбинаций!")
    print(f"   Рассмотрите возможность:")
    print(f"   - Уменьшения количества значений параметров")
    print(f"   - Использования RandomSearchStrategy с max_iterations=1000")
    print(f"   - Разделения оптимизации на этапы")

print("\n" + "=" * 50)



📊 АНАЛИЗ ПАРАМЕТРОВ ОПТИМИЗАЦИИ
📈 Будет протестировано: 35,569 свечей
📅 Период данных: 2025-01-15 - 2025-10-14

🔢 ПАРАМЕТРЫ ОПТИМИЗАЦИИ:
  KATR: 7 значений [2, 3, 4, 5, 6, 7, 8]
  PerATR: 3 значений [5, 20, 80]
  SMA: 4 значений [1, 2, 3, 10]
  MinRA: 1 значений [0]
  FlATR: 1 значений [0]
  FlHL: 1 значений [0]

🧮 РАСЧЕТ КОМБИНАЦИЙ:
  Всего параметров: 6
  Комбинаций: 84
  Формула: 7 × 3 × 4 × 1 × 1 × 1 = 84

⏱️ ОЦЕНКА ВРЕМЕНИ:
  ~0.1 сек на комбинацию
  Общее время: 8.4 сек (0.1 мин)



In [18]:
import time
# === ЗАПУСКАЕМ ОПТИМИЗАЦИЮ ===
print("🚀 Начинаем оптимизацию...")
start_time = time.time()

run_dir = executor.run(df)  # фиксируем одиночный воркер

end_time = time.time()
actual_time = end_time - start_time

# === РЕЗУЛЬТАТЫ ===
results = list(executor.results_store)
print(f"✅ Завершено! Получено {len(results)} результатов")
print(f"⏱️ Фактическое время: {actual_time:.1f} сек ({actual_time/60:.1f} мин)")
print(f"📊 Скорость: {len(results)/actual_time:.1f} комбинаций/сек")
best_result = max(results, key=lambda r: r.metrics.get('total_return', 0))
print(f"🏆 Лучший результат: {best_result.parameters}")
print(f"📊 Метрики: {best_result.metrics}")
print(f"📁 Директория запуска: {run_dir}")

🚀 Начинаем оптимизацию...
✅ Завершено! Получено 84 результатов
⏱️ Фактическое время: 98.2 сек (1.6 мин)
📊 Скорость: 0.9 комбинаций/сек
🏆 Лучший результат: {'katr': 2, 'peratr': 5, 'sma': 1, 'minra': 0, 'flatr': 0, 'flhl': 0}
📊 Метрики: {'total_return': -0.0046917202000005265, 'max_drawdown': 0.008462621249999969, 'recovery_factor': -0.5544050786865292, 'sharpe': -0.05926422374746278, 'sortino': -0.07526407954636909, 'n_flips': 484, 'n_trades': 485, 'win_rate': 0.3422680412371134, 'profit_factor': 0.9058461399559486}
📁 Директория запуска: output\optimization\20251022_170041


In [19]:
# === ТАБЛИЦА РЕЗУЛЬТАТОВ ТЕСТИРОВАНИЯ ===
import pandas as pd

# Создаем DataFrame с результатами
results_data = []
for i, result in enumerate(results):
    row = {
        'Индекс': i,
        'KATR': result.parameters.get('katr'),
        'PerATR': result.parameters.get('peratr'), 
        'SMA': result.parameters.get('sma'),
        'MinRA': result.parameters.get('minra'),
        'FlATR': result.parameters.get('flatr'),
        'FlHL': result.parameters.get('flhl'),
        'Total Return': result.metrics.get('total_return', 0),
        'Max Drawdown': result.metrics.get('max_drawdown', 0),
        'Recovery Factor': result.metrics.get('recovery_factor', 0),
        'Sharpe': result.metrics.get('sharpe', 0),
        'Sortino': result.metrics.get('sortino', 0),
        'Win Rate': result.metrics.get('win_rate', 0),
        'Profit Factor': result.metrics.get('profit_factor', 0),
        'N Trades': result.metrics.get('n_trades', 0)
    }
    results_data.append(row)

# Создаем DataFrame и сортируем по Total Return
results_df = pd.DataFrame(results_data)
results_df = results_df.sort_values('Total Return', ascending=False).reset_index(drop=True)

print("📊 ТАБЛИЦА РЕЗУЛЬТАТОВ ОПТИМИЗАЦИИ")
print("=" * 80)
print(f"Всего результатов: {len(results_df)}")
print(f"Лучший результат (индекс {results_df.index[0]}):")
print(f"  Total Return: {results_df.iloc[0]['Total Return']:.4f}")
print(f"  Max Drawdown: {results_df.iloc[0]['Max Drawdown']:.4f}")
print(f"  Sharpe: {results_df.iloc[0]['Sharpe']:.4f}")
print("=" * 80)

# Отображаем топ-10 результатов
print("\n🏆 ТОП-10 РЕЗУЛЬТАТОВ:")
display(results_df.head(10).round(4))


📊 ТАБЛИЦА РЕЗУЛЬТАТОВ ОПТИМИЗАЦИИ
Всего результатов: 84
Лучший результат (индекс 0):
  Total Return: -0.0047
  Max Drawdown: 0.0085
  Sharpe: -0.0593

🏆 ТОП-10 РЕЗУЛЬТАТОВ:


,Индекс,KATR,PerATR,SMA,MinRA,FlATR,FlHL,Total Return,Max Drawdown,Recovery Factor,Sharpe,Sortino,Win Rate,Profit Factor,N Trades
0,0,2,5,1,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
1,53,6,20,2,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
2,61,7,5,2,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
3,60,7,5,1,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
4,59,6,80,10,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
5,58,6,80,3,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
6,57,6,80,2,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
7,56,6,80,1,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
8,55,6,20,10,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485
9,54,6,20,3,0,0,0,-0.0047,0.0085,-0.5544,-0.0593,-0.0753,0.3423,0.9058,485


In [20]:
# === ПЕРЕЗАГРУЗКА ФУНКЦИЙ (ВАЖНО!) ===
# Перезагружаем исправленную функцию build_indicator
def build_indicator(df, params, *, verbose=True):
    """Создаем индикатор с параметрами (ИСПРАВЛЕННАЯ ВЕРСИЯ)"""
    vm_ind = IndicatorRegistry.get("VolatilityMedian")

    # Фильтруем только нужные параметры для VolatilityMedian
    # Параметры для VolatilityMedian: KATR, PerATR, SMA, MinRA, FlATR, FlHL
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL'
    }

    indicator_params = {}
    for param_key, param_value in (params or {}).items():
        normalized_key = str(param_key).lower()
        mapped_key = param_mapping.get(normalized_key)
        if mapped_key and param_value is not None:
            indicator_params[mapped_key] = param_value

    if verbose:
        print(f"🔧 Параметры для индикатора: {indicator_params}")
    return vm_ind.calculate(df, **indicator_params)

print("✅ Функция build_indicator перезагружена с исправлениями!")


def _ensure_series(signals, index):
    """Приводим сигналы к Series с корректным индексом"""
    if hasattr(signals, "index"):
        return signals
    import pandas as pd
    return pd.Series(signals, index=index)


# === ПОСТРОЕНИЕ ЭКВИТИ ПО ИНДЕКСУ РЕЗУЛЬТАТА ===
def build_equity_for_result(result_index, *, verbose=True):
    """Строит эквити для результата по индексу"""
    if result_index >= len(results):
        print(f"❌ Ошибка: Индекс {result_index} превышает количество результатов ({len(results)})")
        return None

    # Получаем результат по индексу
    target_result = results[result_index]
    print(f"📊 Строим эквити для результата #{result_index}")
    print(
        "Метрики: Total Return={:.4f}, Max DD={:.4f}".format(
            target_result.metrics.get('total_return', 0),
            target_result.metrics.get('max_drawdown', 0)
        )
    )

    # Создаем конфигурацию сигнала для этого результата
    signal_config = create_signal_config(target_result.parameters)

    # Строим индикатор
    indicator_data = build_indicator(df, target_result.parameters, verbose=verbose)
    indicator_series = getattr(signal_config, 'indicator', None) or indicator_data
    if indicator_series is not None and hasattr(indicator_series, "reindex"):
        indicator_series = indicator_series.reindex(df.index).ffill()

    # Генерируем сигналы на основе конфигурации
    raw_signals = signal_config.signal.generate(df, indicator_series, **signal_config.signal_params)
    signals = _ensure_series(raw_signals, df.index)

    # Создаем бэктест движок и применяем параметры из конфигурации
    backtest_engine = BacktestEngine()
    backtest_params = dict(getattr(signal_config, 'backtest_params', {}))
    backtest_result = backtest_engine.run(df, signals, **backtest_params)

    # Извлекаем эквити
    equity = backtest_result.get('equity')
    if equity is not None:
        print(f"✅ Эквити построена: {len(equity)} точек")
        print(
            "Повторный бэктест: Total Return={:.4f}, Max DD={:.4f}".format(
                backtest_result.get('total_return', 0),
                backtest_result.get('max_drawdown', 0)
            )
        )
        return equity

    print("❌ Ошибка: Не удалось получить эквити из результата бэктеста")
    return None


# === ИНТЕРАКТИВНОЕ ПОСТРОЕНИЕ ЭКВИТИ ===
# Введите индекс результата для построения эквити (0-83)
result_index = 0  # ← ИЗМЕНИТЕ ЭТО ЗНАЧЕНИЕ НА НУЖНЫЙ ИНДЕКС
verbose = True

equity = build_equity_for_result(result_index, verbose=verbose)

if equity is not None:
    print(f"📈 Эквити готова для отображения: {len(equity)} точек")
    print(f"Начальное значение: {equity.iloc[0]:.2f}")
    print(f"Конечное значение: {equity.iloc[-1]:.2f}")
    print(f"Максимум: {equity.max():.2f}")
    print(f"Минимум: {equity.min():.2f}")
else:
    print("❌ Не удалось построить эквити")


✅ Функция build_indicator перезагружена с исправлениями!
📊 Строим эквити для результата #0
Параметры: {'katr': 2, 'peratr': 5, 'sma': 1, 'minra': 0, 'flatr': 0, 'flhl': 0}
Метрики: Total Return=-0.0047, Max DD=0.0085
🔧 Параметры для индикатора: {'KATR': 2, 'PerATR': 5, 'SMA': 1, 'MinRA': 0, 'FlATR': 0, 'FlHL': 0}


TypeError: BacktestEngine.run() got an unexpected keyword argument 'data'

In [21]:
# === ОТОБРАЖЕНИЕ ЭКВИТИ С BOKEH ===
if equity is not None:
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import HoverTool
    
    output_notebook()
    
    # Создаем график
    p = figure(
        title=f'Equity Curve (Результат #{result_index})', 
        x_axis_type='datetime', 
        width=900, 
        height=420, 
        tools='pan,wheel_zoom,box_zoom,reset,save'
    )
    
    # Добавляем линию эквити
    p.line(equity.index, equity.values, line_width=2, color='#2962FF', legend_label='Equity')
    
    # Настраиваем hover
    p.add_tools(HoverTool(
        tooltips=[
            ('Время', '@x{%F %T}'), 
            ('Эквити', '@y{0.00}')
        ], 
        formatters={'@x': 'datetime'}
    ))
    
    # Настройки графика
    p.legend.location = 'top_left'
    p.grid.grid_line_alpha = 0.25
    p.xaxis.axis_label = 'Время'
    p.yaxis.axis_label = 'Эквити'
    
    # Показываем график
    show(p)
    
    # Дополнительная статистика
    print(f"\n📊 СТАТИСТИКА ЭКВИТИ:")
    print(f"  Начальная эквити: {equity.iloc[0]:.2f}")
    print(f"  Конечная эквити: {equity.iloc[-1]:.2f}")
    print(f"  Общая доходность: {(equity.iloc[-1] / equity.iloc[0] - 1) * 100:.2f}%")
    print(f"  Максимальная эквити: {equity.max():.2f}")
    print(f"  Минимальная эквити: {equity.min():.2f}")
    print(f"  Максимальная просадка: {((equity.cummax() - equity) / equity.cummax()).max() * 100:.2f}%")
else:
    print("❌ Эквити не построена. Сначала выполните предыдущую ячейку.")


NameError: name 'equity' is not defined

In [ ]:
# ## спящая ячейка для отображения графика
# df_chart = df.copy()
# df_chart = df_chart.rename(columns={
#     'OPEN': 'open',
#     'HIGH': 'high',
#     'LOW': 'low',
#     'CLOSE': 'close',
#     'VOL': 'volume'
# })

# print(f"Всего данных для отображения: {len(df_chart)} баров")

# # Вариант 1: Отобразить все данные (может быть медленно)
# chart = JupyterChart()
# chart.set(df_chart)
# chart.load()

In [ ]:

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool



output_notebook()
p = figure(title='Equity (Bokeh)', x_axis_type='datetime', width=900, height=420, tools='pan,wheel_zoom,box_zoom,reset,save')
p.line(equity.index, equity.values, line_width=2, color='#2962FF', legend_label='Equity')
p.add_tools(HoverTool(tooltips=[('time', '@x{%F %T}'), ('equity', '@y')], formatters={'@x': 'datetime'}))
p.legend.location = 'top_left'
p.grid.grid_line_alpha = 0.25
show(p)
